---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2018</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1>Issued Monday, 3/26/2018; Due Monday, 4/5/2018</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [2]:
# Replace the following string values with the requested information
class Student:
    first = ""
    last = ""
    onyen = ""
    pid = ""

---
**Problem #1:** Find the longest common subsequence (LCS) between the first 100 bases and the last 100 bases of the [*Salmonella Typhimurium*](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.fa) genome. Report both the length of the LCS as well as the subsequence. Do not include the *+* sign at the start of the genome.

In [180]:
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

header, seq = loadFasta("SalmonellaTyphimurium.fa")
seq = seq[0]
front = seq[1:101]
back = seq[len(seq)-100:len(seq)]

from numpy import *
def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in xrange(1,len(v)+1):
        for j in xrange(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt
def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    if (b[i,j] == 3):
        result = LCS(b,v,i-1,j-1)
        result = result + v[i-1]
        return result
    else:
        if (b[i,j] == 2):
            return LCS(b,v,i,j-1)
        else:
            return LCS(b,v,i-1,j)
    
v = front
w = back
s, b = findLCS(front,back)
print LCS(b,v,b.shape[0]-1,b.shape[1]-1)
print len(LCS(b,v,b.shape[0]-1,b.shape[1]-1))

GGCTCTGGGAAGAGATAGAAGGGGGTTGTTGAAAAAAAATAACAAATAACGTGGAATTTAATT
63


---
**Problem #2:** Find the length of the longest *embedded* common subsequence in the following: 

    "GTACAAAAACAACACAAGAAAAGACAATAAAATTCCATTCGATTCTATTCAATTGGTACA"

The longest embedded common subsequence is the longest common subsequence within in a sequence for which no symbol matches itself. As an example, consider "TAGACTAAG". The longest common subsequence is the sequence itself, "TAGACTAAG", but the longest *embedded* common subsequence is "TAG". 

In [179]:
from numpy import *
def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in xrange(1,len(v)+1):
        for j in xrange(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1] and i!=j):
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

def LCS(b,v,i,j):
    if ((i == 0) or (j == 0)):
        return ''
    if (b[i,j] == 3):
        result = LCS(b,v,i-1,j-1)
        result = result + v[i-1]
        return result
    else:
        if (b[i,j] == 1):
            return LCS(b,v,i-1,j)
        else:
            return LCS(b,v,i,j-1)
    
v = "GTACAAAAACAACACAAGAAAAGACAATAAAATTCCATTCGATTCTATTCAATTGGTACA"
w = "GTACAAAAACAACACAAGAAAAGACAATAAAATTCCATTCGATTCTATTCAATTGGTACA"
s, b = findLCS(v,w)

#for i in xrange(len(s)):
    #print "%12s %-20s" % ('' if i else 'backtrack =', str(b[i]))
#for i in xrange(len(s)):
    #print "%10s %-20s " % ('' if i else 'score =', str(s[i]))
print LCS(b,v,b.shape[0]-1,b.shape[1]-1)
print len(LCS(b,v,b.shape[0]-1,b.shape[1]-1))

AAAAAACACAAAAAAAAAAAAAATTCATTCATTCTTTCA
39


---
**Problem #3:** On your midterm you were asked to find all 16-peptide chains whose molecular weight equals a given target (1024) using the following list of residues ordered by their weights. Most of the algorithms proposed were time-consuming with many possible answers.

```python
Residues = [
    (186, 'W'), (163, 'Y'), (156, 'R'), (147, 'F'), (137, 'H'), 
    (131, 'M'), (129, 'E'), (128, '[K/Q]'), (115, 'D'), (114, 'N'),
    (113, '[I/L]'), (103, 'C'), (101, 'T'), (99, 'V'), (97, 'P'), 
    (87, 'S'), (71, 'A'), (57, 'G')
]
```

Now consider a slightly different problem: find the *length* of the *shortest peptide chain* whose molecular weight matches a target. This problem can be solved effciently using *dynamic programming*. In order to see how, consider an analogy between finding the shortest peptide combination of a given weight, and the problem of making *exact change* from an arbitrary set of coinage. 

In the space below given a reccurance relation for the dynamic program for finding the shortest combination of peptides, *DPShortestPeptideLength(target, Residues)*.

---
**Problem #4:** It is inefficient to create and fully populate a table for *DPShortestPeptideLength(target, Residues)* where every molecular weight less than your given target, as you would for computing exact change. Suggest a more efficient means for representing and populating the memoization of intermediate results needed for *DPShortestPeptideLength(target, Residues)*.

**Problem #5: Programming Problem**

Implement a *dynamic programming* solution for *DPShortestPeptideCombination(target, Residues)*, which is similar to the task described in Problem #4, except finds a shortest peptide chain whose molecular weight matches a target. In the space provided below write your function and test it by generating a shortest peptide combination whose molecular weight is 555, 1024, and 2048.

In [181]:
# Enter your answer here
Residues = [
    (186, 'W'), (163, 'Y'), (156, 'R'), (147, 'F'), (137, 'H'), 
    (131, 'M'), (129, 'E'), (128, '[K/Q]'), (115, 'D'), (114, 'N'),
    (113, '[I/L]'), (103, 'C'), (101, 'T'), (99, 'V'), (97, 'P'), 
    (87, 'S'), (71, 'A'), (57, 'G')
]
def shortestPeptideLength(r, weight):
    minimum_length = [0]
    sequence_composition = [[]]

    for i in xrange(1, weight + 1):
        best = 100000
        best_sequence = None

        for j in r:
            if i - j > -1 and minimum_length[i - j] + 1 < best:
                best = minimum_length[i - j] + 1
                best_sequence = sequence_composition[i - j] + [j]

        minimum_length.append(best)
        sequence_composition.append(best_sequence)

        if i == weight:
            sequence = ""
            for k in range(len(best_sequence)):
                for z in range(len(Residues)):
                    if best_sequence[k] == Residues[z][0]:
                        sequence += Residues[z][1]
            print sequence, len(sequence)
r = []
for i in range(len(Residues)):
    r.append(Residues[i][0])

shortestPeptideLength(r, 555)
shortestPeptideLength(r, 1024)
shortestPeptideLength(r, 2048)



CCYW 4
FRYWWW 6
GMWWWWWWWWWW 12


In [177]:
def DPShortestPeptideLength(target, Residues):
    c = []
    for i in range(len(Residues)):
        c.append(Residues[i][0])
    chain = [[0 for i in xrange(len(c))]]
    print chain
    for m in xrange(1,target+1):
        bestChain = m+1
        for i in xrange(len(c)):
            if (m >= c[i]):
                thisChain = [x for x in chain[m - c[i]]]
                thisChain[i] += 1
                if (sum(thisChain) < bestChain):
                    chain[m:m] = [thisChain]
                    bestChain = sum(thisChain)
    print bestChain
    summ = 0
    peptide = ""
    print chain[target]

    for i in range(len(chain[target])):
        summ += chain[target][i]
        for j in range(chain[target][i]):
            peptide += Residues[i][1]
    return summ,peptide
print DPShortestPeptideLength(555, Residues)
print DPShortestPeptideLength(1024, Residues)
print DPShortestPeptideLength(203, Residues)

CCYW 4
FRYWWW 6
GMWWWWWWWWWW 12


Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set